## 데이터 수집


In [16]:
from sklearn.datasets import load_diabetes
data_diabetes = load_diabetes()
features, label = data_diabetes.data, data_diabetes.target

In [17]:
features.shape, label.shape

((442, 10), (442,))


## 데이터 전처리
- 데이터 분석

## 데이터 분할
- train, test, validation

In [18]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_label, test_label = train_test_split(features, label, test_size=0.2, random_state=42)

In [19]:
train_features.shape, test_features.shape, train_label.shape, test_label.shape

((353, 10), (89, 10), (353,), (89,))

In [20]:
type(train_features), type(test_features), type(train_label), type(test_label)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [21]:
import torch

In [22]:
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)
train_label_tensor = torch.tensor(train_label, dtype=torch.float32).view(-1,1) # 행만 있는 경우 행열로 변환 해줌
# test_features_tensor = torch.tensor(test_features)
# test_label_tensor = torch.tensor(test_label)
type(train_features_tensor), type(train_label_tensor)

(torch.Tensor, torch.Tensor)

In [23]:
train_features_tensor.shape, train_label_tensor.shape

(torch.Size([353, 10]), torch.Size([353, 1]))

## 모델 학습



In [24]:
torch.nn.Module?

Init signature: torch.nn.Module(*args, **kwargs) -> None
Docstring:     
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing them to be nested in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self) -> None:
            super().__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            return F.relu(self.conv2(x))

Submodules assigned in this way will be registered, and will also have their
parameters converted when you call :meth:`to`, etc.

.. note::
    As per the example above, an ``__init__()`` call to the parent class
    must be made before assignment on the child.

:ivar training: Boolean represents whether this module is in training o

In [25]:
# Neural Network model linear regression
# model, loss function, optimizer function
class LinearRegressionNNM(torch.nn.Module) :
    def __init__(self, input_dim): # input : feature 의 열 갯수
        super(LinearRegressionNNM, self).__init__()
        # super(self).__init__()
        self.hidden_1 = torch.nn.Linear(input_dim, 64) # input_dim : feature 수 10개, output_dim : 출력 수 64 개
        self.hidden_2 = torch.nn.Linear(64, 32) # input_dim : 이전 layer output_dim 수 64개, output_dim : 출력 수 32 개
        self.hidden_3 = torch.nn.Linear(32, 16) # input_dim : 이전 layer output_dim 수 32, output_dim : 출력 수 16 개
        self.output = torch.nn.Linear(16, 1) # input_dim : 이전 layer output_dim 수 16, output_dim : 출력 수 1 개 Linear 니까
        self.relu = torch.nn.ReLU() # activation func ReLU 쓸거임. 

    def forward(self, x):
        x = self.relu(self.hidden_1(x)) # hidden layer 1 10,64
        x = self.relu(self.hidden_2(x)) # hidden layer 2 64,32
        x = self.relu(self.hidden_3(x)) # hidden layer 3 32,16
        out = self.output(x) # hidden layer 4 16,1 result 
        
        return out


In [26]:
train_features_tensor.shape[1], train_label_tensor.shape[1]

(10, 1)

In [27]:
model = LinearRegressionNNM(train_features_tensor.shape[1])

In [28]:
model

LinearRegressionNNM(
  (hidden_1): Linear(in_features=10, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [29]:
criterion = torch.nn.MSELoss() # Loss function 거의 label 타입에 따라 결정됨
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # Optimizer fuction
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # Optimizer fuction

In [30]:
## 반복 학습
# loss 가 한자리 까지 줄어야 해
# for epoch in range(10):
# for epoch in range(1000):
for epoch in range(10000):
    pred_y = model.forward(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f"epoch : {epoch}, loss : {loss.item()}")
    # print(f"epoch : {epoch}, loss : {loss.item()}")
    



epoch : 0, loss : 29771.736328125


epoch : 1000, loss : 2963.452880859375
epoch : 2000, loss : 2859.1826171875
epoch : 3000, loss : 2772.788818359375
epoch : 4000, loss : 2603.40625
epoch : 5000, loss : 2014.0003662109375
epoch : 6000, loss : 1484.9698486328125
epoch : 7000, loss : 1255.8370361328125
epoch : 8000, loss : 1121.887451171875
epoch : 9000, loss : 1031.7091064453125


In [ ]:
model.state_dict()['hidden_1.weight'].shape, model.state_dict()['hidden_1.bias'].shape

In [ ]:
model.state_dict()['hidden_2.weight'].shape, model.state_dict()['hidden_2.bias'].shape

In [ ]:
model.state_dict()['hidden_3.weight'].shape, model.state_dict()['hidden_3.bias'].shape

In [ ]:
model.state_dict()['output.weight'].shape, model.state_dict()['output.bias'].shape

## 모델 평가



In [31]:
model.eval()

LinearRegressionNNM(
  (hidden_1): Linear(in_features=10, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [32]:
with torch.no_grad(): # 학습 목정이 아닌 평가 목적 위해 고정
    pred_y = model(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor) # 예측도 간은 loss function
    print(f"loss : {loss.item()}")


loss : 948.1812133789062


In [33]:
# 결과 값(예측값, 원래값)
pred_y[0], train_label[0]

(tensor([198.4411]), 144.0)

## 모델 배포

In [34]:
# 모델 weight와 bias 파일로 저장 (checkpoints)
torch.save(model.state_dict(), "linear_regression_nnm.pth")